In [ ]:
def memoize(f):
    """
    caches a function's return value each time it is called,
    if called later with the same arguments, 
    the cached value is returned (not reevaluated)
    """
    memo = {}
    def helper( c, ts ):
        if x not in memo:            
            memo[( c, ts )] = f( c, ts )
        return memo[( c, ts )]
    return helper

In [ ]:
@memoize
def rec_tsp_solve(c, ts):
    if ts:       
        value = min( ( d[lc][c] + rec_tsp_solve( lc, ts - frozenset([lc]) )[0], lc )
                   for lc in ts)
        print(value)
        return value
    else:
        return d[0][c], 0

In [ ]:
def tsp_rec_solve(d):
    def rec_tsp_solve(c, ts):
        assert c not in ts
        if ts:
            return min((d[lc][c] + rec_tsp_solve(lc, ts - set([lc]))[0], lc)
                       for lc in ts)
        else:
            return (d[0][c], 0)
 
    best_tour = []
    c = 0
    cs = set(range(1, len(d)))
    while True:
        l, lc = rec_tsp_solve(c, cs)
        if lc == 0:
            break
        best_tour.append(lc)
        c = lc
        cs = cs - set([lc])
 
    best_tour = tuple(reversed(best_tour))
 
    return best_tour

Get the path of minimum length that starts at city 0, passes through a set of cities ts (not including i or 0) in any order and ends at city c. where c represents the target city, ts is the intermediate set of cities and the next-to-last city is returned to allow reconstructing the path. Adding the reconstruction code:

In [ ]:
best_tour = []
c = 0 # starting city (target city)
cs = frozenset( range( 1, len(d) ) ) # city set (rest of the cities)
cs - set({1})

# Dynamic Programming

Dynamic programming is a really useful general technique for solving problems that involves breaking down problems into smaller overlapping sub-problems, storing the results computed from the sub-problems and reusing those results on larger chunks of the problem. This approach is pretty much always more efficent than naive brute-force solutions.
  
While the core ideas behind dynamic programming are actually pretty simple, it turns out that it's fairly challenging to use on non-trivial problems because it's often not obvious how to frame a difficult problem in terms of overlapping sub-problems. We'll try to build solutions to several well-known problems and see how the problems can be decomposed to use dynamic programming.

## Fibonacci Numbers

First we'll look at the problem of computing numbers in the [Fibonacci sequence](https://en.wikipedia.org/wiki/Fibonacci_number).  The problem definition is very simple - each number in the sequence is the sum of the two previous numbers in the sequence.  Or, more formally:

$F_n = F_{n-1} + F_{n-2}$, with $F_0 = 0$ and $F_1 = 1$ as the seed values.

Our solution will be responsible for calculating each of Fibonacci numbers up to some defined limit.  We'll first implement a naive solution that re-calculates each number in the sequence from scratch.

In [ ]:
def fib(n):
    if n == 0:
        return 0
    if n == 1:
        return 1
    
    return fib(n - 1) + fib(n - 2)

def all_fib(n):
    fibs = []
    for i in range(n):
        fibs.append(fib(i))
    
    return fibs

In [ ]:
# naive recursion implementation's running time (around 1 minute)
# %time print(all_fib(40))

In [ ]:
def memoize(f):
    """
    caches a function's return value each time it is called,
    if called later with the same arguments, 
    the cached value is returned (not reevaluated)
    """
    memo = {}
    def helper(x):
        if x not in memo:            
            memo[x] = f(x)
        return memo[x]
    return helper

@memoize
def fib1(n):
    if n == 0:
        return 0
    if n == 1:
        return 1
    
    return fib(n - 1) + fib(n - 2)

In [ ]:
%time print(fib1(40))

In [20]:
import numpy as np
from itertools import combinations

In [2]:
d = np.array( [[ 0, 1, 15, 6 ], [ 2, 0, 7, 3 ], [ 9, 6, 0, 12 ], [ 10, 4, 8, 0 ] ] )

In [32]:
d = np.array( [[ 0, 2, 9, 10 ], [ 1, 0, 6, 4 ], [ 15, 7, 0, 8 ], [ 6, 3, 12, 0 ] ] )

# start from 0
d

array([[ 0,  2,  9, 10],
       [ 1,  0,  6,  4],
       [15,  7,  0,  8],
       [ 6,  3, 12,  0]])

In [53]:
n = d.shape[0]
C = {} # key is the from, to; value is the cost and the from city

# starts from city 0
for k in range( 1, n ):
    C[( k, 0 )] = ( d[k][0], 0 ) 

In [54]:
C

{(1, 0): (1, 0), (2, 0): (15, 0), (3, 0): (6, 0)}

In [55]:
whole_set = set(range( 1, n ))
whole_set

{1, 2, 3}

In [56]:
# 1
for subset in whole_set:
    remaining = whole_set - set({subset})
    for r in remaining:
        C[( subset, r )] = ( d[subset][r] + d[r][0], r )

In [49]:
for subsets in combinations( whole_set, 2 ):
    print(subsets)
    subsets = set(subsets)
    remaining = whole_set - subsets # remaining length = 1
    
    dists
    for subset in subsets:
        dist = d[remaining][subset] + 
        dists.append(dist)
    print(remaining)

(1, 2)
frozenset({3})
(1, 3)
frozenset({2})
(2, 3)
frozenset({1})


In [ ]:
# 2


In [42]:
C

{(1, 0): (1, 0),
 (1, 2): (21, 2),
 (1, 3): (10, 3),
 (2, 0): (15, 0),
 (2, 1): (8, 1),
 (2, 3): (14, 3),
 (3, 0): (6, 0),
 (3, 1): (4, 1),
 (3, 2): (27, 2)}

In [25]:
for subset_size in range( 2, n ):
    for subsets in combinations( range( 1, n ), subset_size ):
        # find the lowest cost to get to this subset
        subsets = set(subsets)
        for subset in subsets:
            s = subsets - set({subset})
            print(s)

{2}
{1}
{3}
{1}
{3}
{2}
{2, 3}
{1, 3}
{1, 2}


In [16]:
C = {}

    # Set transition cost from initial state
for k in range(1, n):
    C[(1 << k, k)] = (d[0][k], 0)
C

{(2, 1): (1, 0), (4, 2): (15, 0), (8, 3): (6, 0)}

In [13]:
held_karp(d)

(21, [0, 1, 3, 2])

In [14]:
1 << 2

4

In [12]:
def held_karp(dists):
    """
    Implementation of Held-Karp, an algorithm that solves the Traveling
    Salesman Problem using dynamic programming with memoization.
    Parameters:
        dists: distance matrix
    Returns:
        A tuple, (cost, path).
    """
    n = len(dists)

    # Maps each subset of the nodes to the cost to reach that subset, as well
    # as what node it passed before reaching this subset.
    # Node subsets are represented as set bits.
    C = {}

    # Set transition cost from initial state
    for k in range(1, n):
        C[(1 << k, k)] = (dists[0][k], 0)

    # Iterate subsets of increasing length and store intermediate results
    # in classic dynamic programming manner
    for subset_size in range(2, n):
        for subset in combinations(range(1, n), subset_size):
            # Set bits for all nodes in this subset
            bits = 0
            for bit in subset:
                bits |= 1 << bit

            # Find the lowest cost to get to this subset
            for k in subset:
                prev = bits & ~(1 << k)

                res = []
                for m in subset:
                    if m == 0 or m == k:
                        continue
                    res.append((C[(prev, m)][0] + dists[m][k], m))
                C[(bits, k)] = min(res)

    # We're interested in all bits but the least significant (the start state)
    bits = (2**n - 1) - 1

    # Calculate optimal cost
    res = []
    for k in range(1, n):
        res.append((C[(bits, k)][0] + dists[k][0], k))
    opt, parent = min(res)

    # Backtrack to find full path
    path = []
    for i in range(n - 1):
        path.append(parent)
        new_bits = bits & ~(1 << parent)
        _, parent = C[(bits, parent)]
        bits = new_bits

    # Add implicit start state
    path.append(0)

    return opt, list(reversed(path))

## DP

http://www.geeksforgeeks.org/travelling-salesman-problem-set-1/

https://mchouza.wordpress.com/2010/11/21/solving-the-travelling-salesman-problem/

http://episte.math.ntu.edu.tw/articles/mm/mm_11_3_02/page3.html

https://github.com/phvargas/TSP-python/blob/master/TSP.py

https://github.com/VedantPro/dynamic_TSP/blob/master/TSP.py

--- 

- [Other People's Notes on Dynamic Programming](http://nbviewer.jupyter.org/github/jdwittenauer/ipython-notebooks/blob/master/notebooks/misc/DynamicProgramming.ipynb)
- [memoization](http://www.python-course.eu/python3_memoization.php)

## Branch and Bound

http://episte.math.ntu.edu.tw/articles/mm/mm_11_3_02/page2.html

http://neo.lcc.uma.es/vrp/solution-methods/branch-and-bound/

https://www.quora.com/What-is-the-branch-and-bound-algorithm-technique

https://github.com/mostafabahri/TSP_Branch-Bound/blob/master/TSP/TSP.py